In [2]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

# Kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng student
cursor.execute('''
    CREATE TABLE student (
        student_id INTEGER PRIMARY KEY,
        name TEXT,
        class TEXT,
        course_id INTEGER,
        score REAL
    )
''')

# Tạo bảng course
cursor.execute('''
    CREATE TABLE course (
        id INTEGER PRIMARY KEY,
        course_name TEXT
    )
''')

# Chèn dữ liệu vào bảng student
students_data = [
    (1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7),
    (2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2),
    (3, 'Pham Van Nam', 'Toan Tin', None, 7.9),
    (4, 'Le Thanh Huyen', 'Toan Tin', 20, 7.2),
    (5, 'Vu Quoc Anh', 'May Tinh', 24, 8.0),
    (6, 'Dang Thuy Linh', 'May Tinh', 24, 5.5),
    (7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2),
    (8, 'Ho Ngoc Mai', 'Toan Tin', 20, 8.8),
    (9, 'Duong Huu Phuc', 'Kinh Te', None, 7.2),
    (10, 'Cao Thi Hanh', 'May Tinh', None, 7.0)
]
cursor.executemany("INSERT INTO student VALUES (?, ?, ?, ?, ?)", students_data)

# Chèn dữ liệu vào bảng course
courses_data = [(12, 'Giai tich'), (34, 'Thong ke'), (26, 'Tin hoc')]
cursor.executemany("INSERT INTO course VALUES (?, ?)", courses_data)

conn.commit()

In [3]:
# 1. Tích Decartes (Cross Join)
df_cross = pd.read_sql_query("""
    SELECT * FROM student CROSS JOIN course;
""", conn)
print("\nTích Decartes:")
print(df_cross.head())



Tích Decartes:
   student_id               name     class  course_id  score  id course_name
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  26     Tin hoc
2           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  34    Thong ke
3           2       Tran Thi Lan   Kinh Te       34.0    9.2  12   Giai tich
4           2       Tran Thi Lan   Kinh Te       34.0    9.2  26     Tin hoc


In [4]:
# 2. INNER JOIN
df_inner = pd.read_sql_query("""
    SELECT student.*, course.course_name
    FROM student
    INNER JOIN course ON student.course_id = course.id;
""", conn)
print("\nINNER JOIN:")
print(df_inner)


INNER JOIN:
   student_id               name     class  course_id  score course_name
0           1  Nguyen Minh Hoang  May Tinh         12    6.7   Giai tich
1           2       Tran Thi Lan   Kinh Te         34    9.2    Thong ke
2           7      Bui Tien Dung   Kinh Te         34    9.2    Thong ke


In [5]:
# 3. LEFT JOIN
df_left = pd.read_sql_query("""
    SELECT student.*, course.course_name
    FROM student
    LEFT JOIN course ON student.course_id = course.id;
""", conn)
print("\nLEFT JOIN:")
print(df_left)


LEFT JOIN:
   student_id               name     class  course_id  score course_name
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7   Giai tich
1           2       Tran Thi Lan   Kinh Te       34.0    9.2    Thong ke
2           3       Pham Van Nam  Toan Tin        NaN    7.9        None
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2        None
4           5        Vu Quoc Anh  May Tinh       24.0    8.0        None
5           6     Dang Thuy Linh  May Tinh       24.0    5.5        None
6           7      Bui Tien Dung   Kinh Te       34.0    9.2    Thong ke
7           8        Ho Ngoc Mai  Toan Tin       20.0    8.8        None
8           9     Duong Huu Phuc   Kinh Te        NaN    7.2        None
9          10       Cao Thi Hanh  May Tinh        NaN    7.0        None


In [6]:

# 4. RIGHT JOIN (SQLite không hỗ trợ trực tiếp RIGHT JOIN, ta đảo lại bảng)
df_right = pd.read_sql_query("""
    SELECT student.*, course.course_name
    FROM course
    LEFT JOIN student ON student.course_id = course.id;
""", conn)
print("\nRIGHT JOIN:")
print(df_right)



RIGHT JOIN:
   student_id               name     class  course_id  score course_name
0         1.0  Nguyen Minh Hoang  May Tinh       12.0    6.7   Giai tich
1         NaN               None      None        NaN    NaN     Tin hoc
2         7.0      Bui Tien Dung   Kinh Te       34.0    9.2    Thong ke
3         2.0       Tran Thi Lan   Kinh Te       34.0    9.2    Thong ke


In [7]:

# 5. FULL OUTER JOIN (SQLite không hỗ trợ trực tiếp, cần UNION)
df_full = pd.read_sql_query("""
    SELECT student.*, course.course_name
    FROM student
    LEFT JOIN course ON student.course_id = course.id
    UNION
    SELECT student.*, course.course_name
    FROM student
    RIGHT JOIN course ON student.course_id = course.id;
""", conn)
print("\nFULL OUTER JOIN:")
print(df_full)



FULL OUTER JOIN:
    student_id               name     class  course_id  score course_name
0          NaN               None      None        NaN    NaN     Tin hoc
1          1.0  Nguyen Minh Hoang  May Tinh       12.0    6.7   Giai tich
2          2.0       Tran Thi Lan   Kinh Te       34.0    9.2    Thong ke
3          3.0       Pham Van Nam  Toan Tin        NaN    7.9        None
4          4.0     Le Thanh Huyen  Toan Tin       20.0    7.2        None
5          5.0        Vu Quoc Anh  May Tinh       24.0    8.0        None
6          6.0     Dang Thuy Linh  May Tinh       24.0    5.5        None
7          7.0      Bui Tien Dung   Kinh Te       34.0    9.2    Thong ke
8          8.0        Ho Ngoc Mai  Toan Tin       20.0    8.8        None
9          9.0     Duong Huu Phuc   Kinh Te        NaN    7.2        None
10        10.0       Cao Thi Hanh  May Tinh        NaN    7.0        None


In [8]:
# 6. Cập nhật course_id còn thiếu bằng giá trị hợp lệ
cursor.execute("""
    UPDATE student
    SET course_id = (SELECT id FROM course ORDER BY RANDOM() LIMIT 1)
    WHERE course_id IS NULL;
""")
conn.commit()


In [9]:
# 7. Tính tổng số sinh viên và điểm trung bình của từng lớp
df_class_avg = pd.read_sql_query("""
    SELECT class, COUNT(*) AS total_students, AVG(score) AS avg_score
    FROM student
    GROUP BY class;
""", conn)
print("\nTổng số sinh viên và điểm trung bình từng lớp:")
print(df_class_avg)


Tổng số sinh viên và điểm trung bình từng lớp:
      class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               4   6.800000
2  Toan Tin               3   7.966667


In [10]:
# 8. Tính tổng số sinh viên và điểm trung bình từng môn học
df_course_avg = pd.read_sql_query("""
    SELECT course_name, COUNT(*) AS total_students, AVG(score) AS avg_score
    FROM student
    INNER JOIN course ON student.course_id = course.id
    GROUP BY course_name;
""", conn)
print("\nTổng số sinh viên và điểm trung bình từng môn:")
print(df_course_avg)


Tổng số sinh viên và điểm trung bình từng môn:
  course_name  total_students  avg_score
0   Giai tich               4        7.2
1    Thong ke               2        9.2


In [11]:
# 9. Xếp hạng sinh viên theo điểm
df_rank = pd.read_sql_query("""
    SELECT name, score, RANK() OVER (ORDER BY score DESC) AS rank
    FROM student;
""", conn)
print("\nXếp hạng sinh viên theo điểm:")
print(df_rank)



Xếp hạng sinh viên theo điểm:
                name  score  rank
0       Tran Thi Lan    9.2     1
1      Bui Tien Dung    9.2     1
2        Ho Ngoc Mai    8.8     3
3        Vu Quoc Anh    8.0     4
4       Pham Van Nam    7.9     5
5     Le Thanh Huyen    7.2     6
6     Duong Huu Phuc    7.2     6
7       Cao Thi Hanh    7.0     8
8  Nguyen Minh Hoang    6.7     9
9     Dang Thuy Linh    5.5    10


In [12]:
# 10. Thêm cột graduation_date
cursor.execute("ALTER TABLE student ADD COLUMN graduation_date DATETIME;")
conn.commit()


In [16]:

# Cập nhật graduation_date dựa trên điểm số
for row in cursor.execute("SELECT student_id, score FROM student;"):
    student_id, score = row
    if score is not None:
        grad_date = datetime.now() + timedelta(days=int(score * 10))
        # Cập nhật graduation_date cho tất cả sinh viên
    cursor.execute("""
    UPDATE student
    SET graduation_date = datetime('now', '+' || (score * 10) || ' days')
    WHERE score IS NOT NULL;
""")

conn.commit()


conn.commit()

# Kiểm tra lại bảng student
df_student = pd.read_sql_query("SELECT * FROM student", conn)
print(df_student)


   student_id               name     class  course_id  score  \
0           1  Nguyen Minh Hoang  May Tinh         12    6.7   
1           2       Tran Thi Lan   Kinh Te         34    9.2   
2           3       Pham Van Nam  Toan Tin         12    7.9   
3           4     Le Thanh Huyen  Toan Tin         20    7.2   
4           5        Vu Quoc Anh  May Tinh         24    8.0   
5           6     Dang Thuy Linh  May Tinh         24    5.5   
6           7      Bui Tien Dung   Kinh Te         34    9.2   
7           8        Ho Ngoc Mai  Toan Tin         20    8.8   
8           9     Duong Huu Phuc   Kinh Te         12    7.2   
9          10       Cao Thi Hanh  May Tinh         12    7.0   

       graduation_date  
0  2025-05-18 06:38:16  
1  2025-06-12 06:38:16  
2  2025-05-30 06:38:16  
3  2025-05-23 06:38:16  
4  2025-05-31 06:38:16  
5  2025-05-06 06:38:16  
6  2025-06-12 06:38:16  
7  2025-06-08 06:38:16  
8  2025-05-23 06:38:16  
9  2025-05-21 06:38:16  
